## Setup

In [1]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

In [2]:
import torch
import torch.nn as nn
from transformers import (
    PretrainedConfig,
    PreTrainedModel,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding,
    EarlyStoppingCallback
)
from datasets import load_dataset
from transformers import AutoTokenizer
from pathlib import Path
import numpy as np
import evaluate
import csv

# Create an HF-compatible PyTorch Model

Read ["Customizing models"](https://huggingface.co/docs/transformers/custom_models) or search online for more info on HF custom models.

Remember this time we are not using CNNs/UNets, but RNNs with text.

In [3]:
# @title Define the config class


class LSTMConfig(PretrainedConfig):
    model_type = "custom_raw_lstm"

    def __init__(
        self,
        vocab_size=30522,  # Default matching DistilBERT (change to match your tokenizer if needed)
        embedding_dim=64,
        hidden_dim=128,    # Size of the "Memory" vector h_t // Era 128
        n_classes=6,       # AG News has 4 classes (again, change if using different dataset)
        dropout=0.2,
        **kwargs
    ):
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.n_classes = n_classes
        self.dropout = dropout
        super().__init__(**kwargs)

In [4]:
# @title Define a recurrent layer and text classification model


# To separate the complexity from the sequence classification task from the complexity of
# using recursion in our network, let's divide the recursion in one class (`LSTMLayer`)
# and the text classification model in another (`TextClassificationRawLSTM`)


# The "layer" we will use inside our model. This class handles recursion so we don't need to worry
# about it in the model class
class LSTMLayer(nn.Module):
    """
    Manual LSTM implementation to expose the time loop and gates.
    """
    def __init__(self, input_size, hidden_size):
        super().__init__()
        self.hidden_size = hidden_size
        self.x2h = nn.Linear(input_size, 4 * hidden_size)
        self.h2h = nn.Linear(hidden_size, 4 * hidden_size)

    def forward(self, x, attention_mask=None):
        batch_size, seq_len, _ = x.size()
        h_t = x.new_zeros((batch_size, self.hidden_size))
        c_t = x.new_zeros((batch_size, self.hidden_size))

        outputs = []
        for t in range(seq_len):
            x_t = x[:, t, :]
            gates = self.x2h(x_t) + self.h2h(h_t)
            i_gate, f_gate, g_gate, o_gate = gates.chunk(4, dim=1)

            i_gate = torch.sigmoid(i_gate)
            f_gate = torch.sigmoid(f_gate)
            g_gate = torch.tanh(g_gate)
            o_gate = torch.sigmoid(o_gate)

            next_c = f_gate * c_t + i_gate * g_gate
            next_h = o_gate * torch.tanh(next_c)

            if attention_mask is not None:
                mask_t = attention_mask[:, t].unsqueeze(1).type_as(next_h)
                h_t = mask_t * next_h + (1.0 - mask_t) * h_t
                c_t = mask_t * next_c + (1.0 - mask_t) * c_t
            else:
                h_t = next_h
                c_t = next_c
            outputs.append(h_t)

        return torch.stack(outputs, dim=1)


class TextClassificationRawLSTM(PreTrainedModel):
    config_class = LSTMConfig

    def __init__(self, config):
        super().__init__(config)
        self.config = config

        self.embedding = nn.Embedding(config.vocab_size, config.embedding_dim, padding_idx=0)
        self.lstm_block = LSTMLayer(
            input_size=config.embedding_dim,
            hidden_size=config.hidden_dim,
        )
        self.dropout = nn.Dropout(config.dropout)
        self.classifier = nn.Linear(config.hidden_dim, config.n_classes)
        self.loss_fn = nn.CrossEntropyLoss()

        self.all_tied_weights_keys = []
        self._tied_weights_keys = []
        self.post_init()

    @property
    def dummy_inputs(self):
        return {"input_ids": torch.tensor([[0, 1]]), "attention_mask": torch.tensor([[1, 1]])}

    def _check_and_adjust_experts_implementation(self, experts_implementation):
        return experts_implementation

    def forward(self, input_ids=None, attention_mask=None, labels=None, **kwargs):
        x = self.embedding(input_ids)  # [Batch, Seq, Emb]
        lstm_output = self.lstm_block(x, attention_mask=attention_mask)  # [Batch, Seq, Hidden]

        if attention_mask is not None:
            sequence_lengths = attention_mask.sum(dim=1) - 1
            batch_size = input_ids.shape[0]
            last_hidden_states = lstm_output[torch.arange(batch_size, device=x.device), sequence_lengths]
        else:
            last_hidden_states = lstm_output[:, -1, :]

        last_hidden_states = self.dropout(last_hidden_states)
        logits = self.classifier(last_hidden_states)

        loss = None
        if labels is not None:
            loss = self.loss_fn(logits, labels)

        return {"loss": loss, "logits": logits}

# Setup Training

In [5]:
# @title Define Data Pipeline (Emotion CSV)

from pathlib import Path

# Use DistilBERT tokenizer just for the vocab mapping
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Resolve project root whether notebook runs from workspace root or from notebooks/ folder
project_root = Path.cwd()
if not (project_root / "data").exists() and (project_root.parent / "data").exists():
    project_root = project_root.parent

train_csv = project_root / "data" / "kaggle_emotion_classification" / "train_large.csv"
if not train_csv.exists():
    raise FileNotFoundError(f"Arquivo nao encontrado: {train_csv}")

# Load labeled data (100% for training)
raw_dataset = load_dataset("csv", data_files={"train": str(train_csv)})["train"]
raw_dataset = raw_dataset.class_encode_column("label")

# Labels are now contiguous [0..N-1] after class_encode_column
num_labels = raw_dataset.features["label"].num_classes
print(f"Total de classes: {num_labels}")

# Show class distribution (important for imbalance diagnosis)
train_labels_np = np.array(raw_dataset["label"])
train_counts = np.bincount(train_labels_np, minlength=num_labels)
print("Distribuicao de classes (train, 100%):", train_counts.tolist())

# Preprocessing
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding=False, max_length=128)

# Remove non-model columns (like raw text/id) to avoid collator issues
columns_to_remove = [col for col in raw_dataset.column_names if col != "label"]
tokenized_dataset = raw_dataset.map(tokenize_function, batched=True, remove_columns=columns_to_remove)
train_dataset = tokenized_dataset

Total de classes: 6
Distribuicao de classes (train, 100%): [103639, 120363, 29214, 48931, 40472, 12584]


In [6]:
# @title Training Setup

# Device-aware knobs for Apple Silicon / CUDA / CPU
is_cuda = torch.cuda.is_available()
is_mps = torch.backends.mps.is_available()

train_batch_size = 64
if is_mps:
    train_batch_size = 64
elif is_cuda:
    train_batch_size = 64

# Setup model (number of classes inferred from dataset)
config = LSTMConfig(
    vocab_size=tokenizer.vocab_size,
    embedding_dim=64,
    hidden_dim=128,
    n_classes=num_labels,
    dropout=0.2
)
model = TextClassificationRawLSTM(config)

# Class weights for imbalance handling (inverse frequency, normalized)
train_counts = np.bincount(np.array(train_dataset["label"]), minlength=num_labels)
class_weights_np = train_counts.sum() / np.maximum(train_counts, 1)
class_weights_np = class_weights_np / class_weights_np.mean()
class_weights = torch.tensor(class_weights_np, dtype=torch.float)
print("Class weights:", class_weights_np.round(4).tolist())

# Setup metrics: Kaggle uses multiclass log loss (cross-entropy on probabilities)
def multiclass_log_loss(logits, labels, eps=1e-15):
    probs = np.exp(logits - logits.max(axis=1, keepdims=True))
    probs = probs / probs.sum(axis=1, keepdims=True)
    probs = np.clip(probs, eps, 1.0 - eps)
    one_hot = np.eye(probs.shape[1])[labels]
    return float(-(one_hot * np.log(probs)).sum(axis=1).mean())

accuracy = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    acc = accuracy.compute(predictions=preds, references=labels)["accuracy"]
    ll = multiclass_log_loss(logits, labels)
    return {"accuracy": acc, "log_loss": ll}

# Weighted Trainer to optimize weighted cross-entropy on imbalanced data
class WeightedTrainer(Trainer):
    def __init__(self, *args, class_weights=None, **kwargs):
        super().__init__(*args, **kwargs)
        self.class_weights = class_weights
        self._cached_cw = None

    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")

        if self.class_weights is not None:
            if self._cached_cw is None or self._cached_cw.device != logits.device:
                self._cached_cw = self.class_weights.to(logits.device)
            loss_fct = nn.CrossEntropyLoss(weight=self._cached_cw)
        else:
            loss_fct = nn.CrossEntropyLoss()

        loss = loss_fct(logits.view(-1, model.config.n_classes), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

# # Setup training loop and args
 # training_args = TrainingArguments(
 #     output_dir="./raw_lstm_emotion",
 #     learning_rate=2e-4,
 #     per_device_train_batch_size=train_batch_size,
 #     per_device_eval_batch_size=max(32, train_batch_size),
 #     num_train_epochs=5,
 #     eval_strategy="no",
 #     save_strategy="epoch",
 #     save_total_limit=2,
 #     load_best_model_at_end=False,
 #     lr_scheduler_type="linear",
 #     warmup_steps=500,
 #     weight_decay=0.01,
 #     max_grad_norm=1.0,
 #     logging_steps=50,
 #     dataloader_num_workers=2,
 #     dataloader_pin_memory=False if is_mps else True,
 #     remove_unused_columns=True,
 #     report_to="none",
 # )

# trainer = WeightedTrainer(
 #     model=model,
 #     args=training_args,
 #     train_dataset=train_dataset,
 #     processing_class=tokenizer,
 #     data_collator=data_collator,
 #     compute_metrics=compute_metrics,
 #     # callbacks=[EarlyStoppingCallback(early_stopping_patience=2)],
 #     class_weights=class_weights,
 # )

# print(f"Device profile -> CUDA: {is_cuda} | MPS: {is_mps} | train_batch_size: {train_batch_size}")

distilbert_model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=num_labels
)

distilbert_training_args = TrainingArguments(
    output_dir="./distilbert_emotion",
    learning_rate=2e-5,  # Lower LR for pre-trained models
    per_device_train_batch_size=16,
    num_train_epochs=3,  # Fewer epochs for fine-tuning
    eval_strategy="no",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=False,
    lr_scheduler_type="linear",
    warmup_ratio=0.1,  # 10% warmup steps
    weight_decay=0.01,
    logging_steps=50,
    dataloader_num_workers=2,
    dataloader_pin_memory=True,
    remove_unused_columns=True,
    report_to="none",
)

trainer = Trainer(
    model=distilbert_model,
    args=distilbert_training_args,
    train_dataset=train_dataset,
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

print(f"DistilBERT carregado: {distilbert_model.num_parameters():,} parametros")

Class weights: [0.3275, 0.282, 1.1617, 0.6936, 0.8385, 2.6968]


Loading weights:   0%|          | 0/100 [00:00<?, ?it/s]

DistilBertForSequenceClassification LOAD REPORT from: distilbert-base-uncased
Key                     | Status     | 
------------------------+------------+-
vocab_transform.weight  | UNEXPECTED | 
vocab_layer_norm.weight | UNEXPECTED | 
vocab_projector.bias    | UNEXPECTED | 
vocab_layer_norm.bias   | UNEXPECTED | 
vocab_transform.bias    | UNEXPECTED | 
classifier.weight       | MISSING    | 
pre_classifier.bias     | MISSING    | 
pre_classifier.weight   | MISSING    | 
classifier.bias         | MISSING    | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
- MISSING	:those params were newly initialized because missing from the checkpoint. Consider training on your downstream task.
warmup_ratio is deprecated and will be removed in v5.2. Use `warmup_steps` instead.


DistilBERT carregado: 66,958,086 parametros


In [7]:
# @title Check device (CPU / CUDA / MPS) + macOS performance knobs

import os

# Allow CPU fallback for ops not implemented on MPS (prevents hard crashes)
os.environ.setdefault("PYTORCH_ENABLE_MPS_FALLBACK", "1")

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"MPS available: {torch.backends.mps.is_available()}")
print(f"MPS built: {torch.backends.mps.is_built()}")
print(f"MPS fallback: {os.environ.get('PYTORCH_ENABLE_MPS_FALLBACK')}")

if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")
    print(f"Number of CUDA devices: {torch.cuda.device_count()}")
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    print("MPS (Metal Performance Shaders) available - using Apple GPU acceleration")
    device = torch.device("mps")
else:
    print("Running on CPU")
    device = torch.device("cpu")

print(f"\nModel will run on: {device}")

PyTorch version: 2.7.1
CUDA available: False
MPS available: True
MPS built: True
MPS fallback: 1
MPS (Metal Performance Shaders) available - using Apple GPU acceleration

Model will run on: mps


In [8]:
# @title Start training

# Self-contained device setup (so this cell works even if device-check cell wasn't run)
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")

trainer.model.to(device)
print(f"Treinando em: {device}")

if device.type == "mps" and hasattr(torch, "mps"):
    torch.mps.empty_cache()

trainer.train()

Treinando em: mps


/Users/pablolemes/Library/Caches/pypoetry/virtualenvs/ia-senac-uc14-frame-generation-project-DJIR5zmm-py3.12/lib/python3.12/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss
50,1.806489
100,1.799777
150,1.795301
200,1.775778
250,1.756622
300,1.730053
350,1.683395
400,1.621189
450,1.568866
500,1.544623


Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

/Users/pablolemes/Library/Caches/pypoetry/virtualenvs/ia-senac-uc14-frame-generation-project-DJIR5zmm-py3.12/lib/python3.12/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

/Users/pablolemes/Library/Caches/pypoetry/virtualenvs/ia-senac-uc14-frame-generation-project-DJIR5zmm-py3.12/lib/python3.12/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

TrainOutput(global_step=66603, training_loss=0.1234905100630534, metrics={'train_runtime': 35558.0758, 'train_samples_per_second': 29.968, 'train_steps_per_second': 1.873, 'total_flos': 1.3029957819182412e+16, 'train_loss': 0.1234905100630534, 'epoch': 3.0})

In [9]:
# @title Save trained model

save_directory = "./raw_lstm_emotion/final_checkpoint"

trainer.save_model(save_directory)
tokenizer.save_pretrained(save_directory)

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

('./raw_lstm_emotion/final_checkpoint/tokenizer_config.json',
 './raw_lstm_emotion/final_checkpoint/tokenizer.json')

# Run Inferences and Calculate Metrics

In [10]:
# @title Load the model checkpoint

# If you are still in the same session that you trained the model in, this is unnecessary.
# This cell enables loading a checkpoint saved earlier.

# Resolve project root whether notebook runs from workspace root or from notebooks/ folder
project_root = Path.cwd()
if not (project_root / "data").exists() and (project_root.parent / "data").exists():
    project_root = project_root.parent

# Candidate checkpoints (first existing path will be used)
candidate_dirs = [
    project_root / "raw_lstm_emotion" / "final_checkpoint",
    project_root / "notebooks" / "raw_lstm_emotion" / "final_checkpoint",
]

save_directory = None
for c in candidate_dirs:
    if c.exists():
        save_directory = str(c)
        break

if save_directory is None:
    raise FileNotFoundError(
        "Nenhum checkpoint final encontrado. Treine e salve o modelo antes de rodar inferência."
    )

print(f"Carregando checkpoint de: {save_directory}")

# Notice we must use our custom class `TextClassificationRawLSTM`, not AutoModel
# loaded_model = TextClassificationRawLSTM.from_pretrained(save_directory)
loaded_tokenizer = AutoTokenizer.from_pretrained(save_directory)

Carregando checkpoint de: /Users/pablolemes/Projects/IA - Senac/UC14/ia_senac_uc14_frame_generation_project/notebooks/raw_lstm_emotion/final_checkpoint


In [11]:
# @title Get training metrics (sanity check on full data)

# When training on 100% of the data there is no validation split.
# This evaluation is only a sanity check on the training set.
eval_trainer = Trainer(
    model=distilbert_model,
    args=TrainingArguments(output_dir="./eval_output", report_to="none"),
    eval_dataset=train_dataset,
    processing_class=loaded_tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

metrics = eval_trainer.evaluate()
print(f"\nTrain Accuracy: {metrics['eval_accuracy']:.2%}")
print(f"Train Log Loss: {metrics['eval_log_loss']:.5f}")
print(f"Train Loss:     {metrics['eval_loss']:.5f}")

/Users/pablolemes/Library/Caches/pypoetry/virtualenvs/ia-senac-uc14-frame-generation-project-DJIR5zmm-py3.12/lib/python3.12/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)



Train Accuracy: 94.75%
Train Log Loss: 0.07461
Train Loss:     0.07461


In [12]:
# @title Run inference on new data


# AG News Labels: 0=World, 1=Sports, 2=Business, 3=Sci/Tech
# Kaggle Emotion Classification Labels: 0=Sadness, 1=Joy, 2=Love, 3=Anger, 4=Fear, 5=Surprise
# label_map = {0: "World", 1: "Sports", 2: "Business", 3: "Sci/Tech"}
label_map = {0: "Sadness", 1: "Joy", 2: "Love", 3: "Anger", 4: "Fear", 5: "Surprise"}

def predict_news_topic(text):
    # Prepare input (tokenize)
    inputs = loaded_tokenizer(text, return_tensors="pt", truncation=True, padding=False)

    # Move inputs to same device as model
    inputs = {k: v.to(distilbert_model.device) for k, v in inputs.items()}

    # Forward pass
    with torch.no_grad():
        outputs = distilbert_model(**inputs)
        logits = outputs["logits"]
        predicted_class_id = torch.argmax(logits, dim=-1).item()

    return label_map[predicted_class_id]

# --- Demo ---
print("\n🤖 INFERENCE DEMO:")
sample_text = "Nvidia shares jumped today after announcing a new AI chip architecture."
prediction = predict_news_topic(sample_text)

print(f"📝 Text: '{sample_text}'")
print(f"🏷️ Predicted Topic: {prediction}")


🤖 INFERENCE DEMO:
📝 Text: 'Nvidia shares jumped today after announcing a new AI chip architecture.'
🏷️ Predicted Topic: Anger


In [13]:
# @title Generate Kaggle submission (test CSV -> class probabilities)

def _resolve_test_csv(root: Path) -> Path:
    candidate = root / "data" / "kaggle_emotion_classification" / "test.csv"
    if candidate.exists():
        return candidate
    candidate_alt = root / "data" / "kaggle_emotion_classification" / "teste.csv"
    if candidate_alt.exists():
        return candidate_alt
    raise FileNotFoundError("Arquivo de teste nao encontrado (test.csv ou teste.csv).")

def _load_test_rows(csv_path: Path):
    with csv_path.open("r", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        if not reader.fieldnames:
            raise ValueError("CSV de teste sem cabecalho.")
        fieldnames = [name.strip() for name in reader.fieldnames]
        id_field = "id" if "id" in fieldnames else None
        text_field = "text" if "text" in fieldnames else None
        if text_field is None:
            non_id_fields = [name for name in fieldnames if name != "id"]
            if not non_id_fields:
                raise ValueError("CSV de teste precisa de uma coluna de texto.")
            text_field = non_id_fields[0]

        ids, texts = [], []
        for idx, row in enumerate(reader):
            row_id = row.get(id_field) if id_field else None
            ids.append(row_id if row_id is not None else str(idx))
            texts.append(row.get(text_field, ""))

    return ids, texts

def _batched(items, batch_size):
    for start in range(0, len(items), batch_size):
        yield items[start:start + batch_size]

# Resolve test file
project_root = Path.cwd()
if not (project_root / "data").exists() and (project_root.parent / "data").exists():
    project_root = project_root.parent

test_csv = _resolve_test_csv(project_root)
ids, texts = _load_test_rows(test_csv)

# Ensure model on the same device and in eval mode
loaded_model = distilbert_model.to(device)
loaded_model.eval()

all_probs = []
batch_size = 128

with torch.no_grad():
    for batch_texts in _batched(texts, batch_size):
        inputs = loaded_tokenizer(
            batch_texts,
            return_tensors="pt",
            truncation=True,
            padding=True,
            max_length=128,
        )
        inputs = {k: v.to(device) for k, v in inputs.items()}
        outputs = loaded_model(**inputs)
        logits = outputs["logits"]
        probs = torch.softmax(logits, dim=-1).cpu().numpy()
        all_probs.extend(probs)

# Sanity checks for Kaggle log-loss submissions
all_probs = np.asarray(all_probs, dtype=np.float64)
if all_probs.shape[1] != num_labels:
    raise ValueError(f"Numero de colunas de probabilidade ({all_probs.shape[1]}) diferente de num_labels ({num_labels}).")

row_sums = all_probs.sum(axis=1)
if not np.allclose(row_sums, 1.0, atol=1e-6):
    raise ValueError("As probabilidades nao somam 1.0 em todas as linhas.")

if np.isnan(all_probs).any() or np.isinf(all_probs).any():
    raise ValueError("Foram encontrados NaN/Inf nas probabilidades.")

# Kaggle expected header: id,prob_0,prob_1,prob_2,prob_3,prob_4,prob_5
expected_num_classes = 6
if num_labels != expected_num_classes:
    raise ValueError(
        f"Esse desafio espera {expected_num_classes} classes, mas o pipeline detectou {num_labels}."
    )
submission_class_columns = [f"prob_{i}" for i in range(expected_num_classes)]

# Write submission CSV
submission_path = project_root / "submission.csv"
with submission_path.open("w", encoding="utf-8", newline="") as f:
    writer = csv.writer(f)
    header = ["id"] + submission_class_columns
    writer.writerow(header)
    for row_id, prob_row in zip(ids, all_probs):
        writer.writerow([row_id] + [f"{p:.8f}" for p in prob_row])

print(f"Arquivo de submissao gerado: {submission_path}")
print(f"Total de linhas: {len(all_probs)}")
print(f"Header da submissao: {["id"] + submission_class_columns}")

Arquivo de submissao gerado: /Users/pablolemes/Projects/IA - Senac/UC14/ia_senac_uc14_frame_generation_project/submission.csv
Total de linhas: 39467
Header da submissao: ['id', 'prob_0', 'prob_1', 'prob_2', 'prob_3', 'prob_4', 'prob_5']
